In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

data = pd.read_csv("D:/Code/imdb_project/movie_metadata.csv")
data = data.dropna()
data.drop_duplicates(keep='first', inplace = True)
data["country"] = data["country"].replace({"USA": 1, "UK": 2})
data["country"] = data["country"].apply(lambda x: 0 if x != 1 and x != 2 else x)
data = data.drop(["color","director_name","actor_2_name", "genres", "actor_1_name","movie_title","content_rating","language",
                 "plot_keywords", "movie_imdb_link","actor_3_name","aspect_ratio"], axis = 1)
zcount = {}
for column in data.columns:
    zcount[column] = (Counter(sorted(data[column]))[0])
zcount.pop('country')
temp = []
for key in zcount.keys():
    if zcount[key] != 0:
        temp.append(key)
for col in temp:
    data[col] = data[col].replace({0.0:data[col].mean()})
len(data.columns)


16

In [ ]:
corr = data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [16]:
data["other_actor_facebook_likes"] = data["actor_3_facebook_likes"] + data["actor_2_facebook_likes"]
data["review_ratio"] = data["num_critic_for_reviews"] / data["num_user_for_reviews"]
data["label"] = data["imdb_score"].apply(lambda x: 1 if x>7 else 0)
data = data.drop(["cast_total_facebook_likes","actor_3_facebook_likes",
                  "actor_2_facebook_likes","num_critic_for_reviews","num_user_for_reviews", "imdb_score"], axis = 1)

len(data.columns)

13

In [ ]:
corr = data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [17]:
data.columns
df = data[['title_year','country','duration', 'budget','gross',
          'review_ratio','num_voted_users','facenumber_in_poster','movie_facebook_likes',
          'director_facebook_likes','actor_1_facebook_likes','other_actor_facebook_likes',
          'label']]
len(df.columns)

13

In [18]:
feature = df.iloc[:,:len(df.columns)-1].values
label = df["label"].values

scaler = StandardScaler()
scaler.fit(feature)
X = scaler.transform(feature)
y = label
scaler.mean_
scaler.scale_
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 5)

In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

classifier = LinearDiscriminantAnalysis()
classifier.fit(x_train, y_train)
print(classifier.score(x_test, y_test))

0.7946308724832215


In [19]:
score = []
accuracy = []
for k in range(1, 100):
    classifier = KNeighborsClassifier(n_neighbors = k, weights = "distance")
    classifier.fit(x_train, y_train)
    score.append(classifier.score(x_test, y_test))
print(f'k = {score.index(max(score))} with {max(score)}')

k = 6 with 0.8214765100671141


In [20]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
#print(classifier.score(x_train, y_train))
print(classifier.score(x_test, y_test))

0.8053691275167785


In [21]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(x_train, y_train)
print(classifier.score(x_test, y_test))

0.8241610738255034


In [32]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier()
tree_model.fit(x_train, y_train)
tree_model.score(x_test, y_test)


0.774496644295302

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameters = {"n_estimators":(100, 200, 300, 400), "criterion":("gini", "entropy"), 
              "max_depth":(100,200,300), "min_samples_split":(100,200)}
classifier= RandomForestClassifier()
    #criterion = 'entropy', max_depth= 200, min_samples_split= 100, n_estimators= 100)
#model = GridSearchCV(classifier, parameters)
#model.fit(x_train, y_train)
#model.best_params_
classifier.fit(x_train, y_train)
accuracy_score(y_test,classifier.predict(x_test))

0.8456375838926175